<a href="https://colab.research.google.com/github/ryandoyle5401/OS_Project1/blob/main/proj1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
%%writefile proj1.c
#include <stdio.h>
#include <stdlib.h>
#include <string.h>
#include <unistd.h>
#include <sys/wait.h>
int main() {

// TODO
// Create a child process for each file
// Read all files, keeping track of how often a specific word appears
// Have functionality to measure time, cpu usage, memory usage (use getrusage)
// Note: first run this using only the 7 child processes. Then run with each
//       process creating at least 2 threads to read the file at different
//       points in parallel.
// For each process, create at least two threads for parallel word counting.
// Use IPC (pipes or shared memory) so the child can communicate their word count
//       to the parent process.

  // Keeps track of how often a specific word appears
  //int counter = 0;
  // Array to store unique words
  char *words[10000];
  // Array to store frequencies
  int frequencies[10000] = {0};
  // Keep track of the number of unique words
  int word_count = 0;

  // Store paths to all 7 files
  char *file_paths[] = {"/content/drive/MyDrive/Colab Notebooks/txtfiles/bib",
    "/content/drive/MyDrive/Colab Notebooks/txtfiles/paper1", "/content/drive/MyDrive/Colab Notebooks/txtfiles/paper2",
    "/content/drive/MyDrive/Colab Notebooks/txtfiles/progc", "/content/drive/MyDrive/Colab Notebooks/txtfiles/progl",
    "/content/drive/MyDrive/Colab Notebooks/txtfiles/progp", "/content/drive/MyDrive/Colab Notebooks/txtfiles/trans"};

  // Total number of child processes
  int NUM_OF_PROCESSES = 7;
  pid_t pid[7];

  // Create pipes for each process
  int pipes[NUM_OF_PROCESSES][2];
  // Initialize pipes
  for (int i = 0; i < NUM_OF_PROCESSES; i++) {
    if (pipe(pipes[i]) == -1) {
      perror("failed to create pipe");
      exit(1);
    }
  }

  // Keyword to keep track of
  char keyword[] = "the";

  for (int i = 0; i < NUM_OF_PROCESSES; i++) {
    pid[i] = fork();
    if (pid[i] == -1) {
      perror("failed to create child process");
      exit(1);
    } else if (pid[i] == 0) {
      // Child process
      FILE *file_ptr;
      file_ptr = fopen(file_paths[i], "r");
      // Check if file exists
      if (file_ptr == NULL) {
        perror("failed to open file");
        exit(1);
      }

      if (file_ptr == NULL) {
        printf("file can't be opened.\n");
        return EXIT_FAILURE;
      }

      int counter = 0;
      char str[256];
      while (fgets(str, sizeof(str), file_ptr) != NULL) {
        // This prints all strings in the file
        //printf("%s", s);

        //What I'm thinking:
        // Create another for loop to loop through the array of words
        // Modify this if statement to check if str is in the array
        // If it's not, insert it into the array at the next empty slot
        // Go to the same spot in the frequencies array and increment the value by 1
        // If it is in the array, go to the same spot in the frequencies array and increment the value by 1
        // Iterate through the words array, see if str is in the words array.
        char *token = strtok(str, " \n\t.,;:!?");
        while (token != NULL) {
          int found = 0;
          for (int j = 0; j < word_count; j++) {
            if (strcmp(token, words[j]) == 0) {
              frequencies[j]++;
              found = 1;
              break;
            }
          }
          if (!found && word_count < 10000) {
            words[word_count] = strdup(token);
            frequencies[word_count]++;
            word_count++;
          }
          token = strtok(NULL, " \n\t.,;:!?");
        }
        //if (strstr(str, keyword) != NULL) {
        //  counter++;
        }
      }
      fclose(file_ptr);

      // Send the count to the parent process via the pipe
      write(pipes[i][1], &counter, sizeof(counter));
      close(pipes[i][1]);  // Close the write end after sending
      exit(0);  // Child process exits after completion
    }
  }

  // Parent process: collect counts from children
  int total_counter = 0;
  for (int i = 0; i < NUM_OF_PROCESSES; i++) {
    close(pipes[i][1]);  // Close the write end of the pipe in the parent

    int child_counter;
    read(pipes[i][0], &child_counter, sizeof(child_counter));  // Read the count from the child
    total_counter += child_counter;

    close(pipes[i][0]);  // Close the read end after reading
  }


  int status;
  pid_t wpid;
  for (int i = 0; i < NUM_OF_PROCESSES; i++) {
    wpid = wait(&status);
  }

  printf("The keyword %s appears %d times", keyword, total_counter);
  return EXIT_SUCCESS;
}

Overwriting proj1.c


In [4]:
%%shell
gcc proj1.c -o proj1
./proj1

proj1.c: In function ‘main’:
proj1.c:105:14: error: ‘file_ptr’ undeclared (first use in this function); did you mean ‘file_paths’?
  105 |       fclose(file_ptr);
      |              ^~~~~~~~
      |              file_paths
proj1.c:105:14: note: each undeclared identifier is reported only once for each function it appears in
proj1.c:108:27: error: ‘counter’ undeclared (first use in this function)
  108 |       write(pipes[i][1], &counter, sizeof(counter));
      |                           ^~~~~~~
proj1.c: At top level:
proj1.c:116:3: error: expected identifier or ‘(’ before ‘for’
  116 |   for (int i = 0; i < NUM_OF_PROCESSES; i++) {
      |   ^~~
proj1.c:116:21: error: expected ‘=’, ‘,’, ‘;’, ‘asm’ or ‘__attribute__’ before ‘<’ token
  116 |   for (int i = 0; i < NUM_OF_PROCESSES; i++) {
      |                     ^
proj1.c:116:42: error: expected ‘=’, ‘,’, ‘;’, ‘asm’ or ‘__attribute__’ before ‘++’ token
  116 |   for (int i = 0; i < NUM_OF_PROCESSES; i++) {
      |                

CalledProcessError: Command 'gcc proj1.c -o proj1
./proj1
' returned non-zero exit status 127.